In [ ]:


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
warnings.filterwarnings("ignore")
# %matplotlib inline

# Loading the dataset Admission_Prediction.csv
df=pd.read_csv("https://raw.githubusercontent.com/srinivasav22/Graduate-Admission-Prediction/master/Admission_Predict_Ver1.1.csv")
df

df.info()

def missing_values_impute(df):
    for i in df:
        if df[i].dtypes=='O':
            mode=df[i].mode()[0]
            df[i].fillna(mode,inplace=True)
        else:
            median=df[i].median()
            df[i].fillna(median,inplace=True)

missing_values_impute(df)

df.isnull().sum()

# Droppping the serial column
df.drop(columns='Serial No.',inplace=True)

# EDA 

# plt.figure(figsize=(15,15))
# plt.suptitle('Univariate Analysis of Numerical Features', fontsize=20, fontweight='bold', alpha=0.8, y=1.)
# for j,i in enumerate(df):
#     plt.subplot(5,3,j+1)
#     sns.kdeplot(x=df[i],shade=True, color='r')
#     plt.tight_layout()

# df.columns

# # Observation :- Research column is a binomially distributed
# # GRE Score,TOEFL Score,SOP, LOR, CGPA are approximately Gaussianly distributed

# df["Research"].value_counts()



# plt.figure(figsize=(10, 15))
# plt.suptitle('Bivariate Analysis of Numerical Features with SalesPrice', fontsize=20, fontweight='bold', alpha=0.8, y=1.)
# columns=df.columns
# for i in range(0, len(columns)):
#     plt.subplot(5, 2, i+1)
#     sns.scatterplot(x=df[columns[i]],y=df["Chance of Admit"],palette="deep")
#     plt.tight_layout()

# Feature columns are





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          500 non-null    int64  
 2   TOEFL Score        500 non-null    int64  
 3   University Rating  500 non-null    int64  
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 35.3 KB


In [ ]:
X=df.iloc[:,:8]

# Target variable (dependent feature)
y=df.iloc[:,7:]
y

X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.25, random_state=100)

scale=StandardScaler()

X_train=pd.DataFrame(scale.fit_transform(X_train),columns=X_train.columns)

X_test=pd.DataFrame(scale.transform(X_test),columns=X_test.columns)

# # Model Bulding

In [ ]:
svr=SVR()

In [ ]:

param={
    "C":[1,2,3],
    "kernel":['linear', 'poly', 'rbf', 'sigmoid'],
    "gamma":[0.001,0.01,0.1,],
    "coef0":[0.0,0.1,],
    "shrinking":[False,True],
    "epsilon":[0.001,0.1,0.2],
    "tol":[0.0001,0.001],
    "cache_size":[200,300]
}

In [ ]:
gscv=GridSearchCV(estimator=svr, param_grid=param)
gscv.fit(X_train,y_train)

GridSearchCV(estimator=SVR(),
             param_grid={'C': [1, 2, 3], 'cache_size': [200, 300],
                         'coef0': [0.0, 0.1], 'epsilon': [0.001, 0.1, 0.2],
                         'gamma': [0.001, 0.01, 0.1],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'shrinking': [False, True], 'tol': [0.0001, 0.001]})

In [ ]:
gscv.best_params_

{'C': 1,
 'cache_size': 200,
 'coef0': 0.0,
 'epsilon': 0.001,
 'gamma': 0.001,
 'kernel': 'linear',
 'shrinking': False,
 'tol': 0.0001}

In [ ]:
print(gscv.best_params_)

{'C': 1, 'cache_size': 200, 'coef0': 0.0, 'epsilon': 0.001, 'gamma': 0.001, 'kernel': 'linear', 'shrinking': False, 'tol': 0.0001}


In [ ]:
dt=DecisionTreeRegressor()

In [ ]:
params={
    'criterion':["squared_error", "friedman_mse", "absolute_error", "poisson"],
    'splitter':["best", "random"],
    'max_depth':[1,2,5,7,10,13,15,17,20,23],
    'min_samples_split':[2,4,7,9,12,15,17,21],
    'min_samples_leaf':[1,3,5,8,10,13,16,18,20],
    'min_weight_fraction_leaf':[0.0,0.1,0.4,0.6,0.8,0.9],
    'max_features':["auto", "sqrt", "log2"],
    'random_state':[33,90,100],
    'max_leaf_nodes':[2,4,6,8,10,15,20],
    'min_impurity_decrease':[0.1,0.3,0.5,0.6,0.9],
    'ccp_alpha':[ 1.77777778e-07, 2.00000000e-07,3.20000000e-07, 4.00000000e-07, 5.33333333e-07, 7.11111111e-07,6.85941043e-05]
}

In [ ]:
gddt=GridSearchCV(estimator=dt,param_grid=params,cv=1000,scoring="r2")

In [ ]:
gddt.fit(X_train,y_train)

In [ ]:
gddt.best_params_

NameError: ignored

In [ ]:
path=dt.cost_complexity_pruning_path(X_train,y_train)
ccp_alpha=path['ccp_alphas']
dt_model=[]
for ccp in ccp_alpha:
    dt_m=DecisionTreeClassifier(ccp_alpha=ccp)
    dt_m.fit(X1,y1)
    dt_model2.append(dt_m)
train_score=[i.score(X1,y1) for i in dt_model2]
test_score=[i.score(X_test,y_test) for i in dt_model2]

NameError: ignored

In [1]:


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
warnings.filterwarnings("ignore")
# %matplotlib inline


In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/itsguptaaman/Datasets/main/Census_Income.csv")
df

,Unnamed: 0,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target
0,0,50,0,83311,0,13,0,0,0,0,0,0,0,13,0,0
1,1,38,0,215646,0,9,0,0,0,0,0,0,0,40,0,0
2,2,53,0,234721,0,7,0,0,0,0,0,0,0,40,0,0
3,3,28,0,338409,0,13,0,0,0,0,1,0,0,40,0,0
4,4,37,0,284582,0,14,0,0,0,0,1,0,0,40,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,32555,27,0,257302,0,12,0,0,0,0,1,0,0,38,0,0
32556,32556,40,0,154374,0,9,0,0,0,0,0,0,0,40,0,1
32557,32557,58,0,151910,0,9,0,0,0,0,1,0,0,40,0,0
32558,32558,22,0,201490,0,9,0,0,0,0,0,0,0,20,0,0


In [3]:
X=df.drop(columns=["target"])
y=df["target"]

In [4]:
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=100)

In [5]:
scale=StandardScaler()
scale.fit(X_train)
X_train=pd.DataFrame(scale.transform(X_train),columns=X_train.columns)
X_train
X_test=pd.DataFrame(scale.transform(X_test),columns=X_test.columns)
X_test

,Unnamed: 0,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,-0.912339,1.280528,0.0,1.917776,0.0,-0.416511,0.0,0.0,0.0,0.0,1.412912,-0.145216,-0.218692,-1.248225,0.0
1,-1.083484,-0.627731,0.0,0.554335,0.0,-0.416511,0.0,0.0,0.0,0.0,1.412912,-0.145216,-0.218692,-0.031995,0.0
2,-1.273444,1.133739,0.0,-0.536039,0.0,-0.026989,0.0,0.0,0.0,0.0,-0.707758,-0.145216,-0.218692,-0.031995,0.0
3,0.353285,-1.214888,0.0,1.428287,0.0,-0.416511,0.0,0.0,0.0,0.0,1.412912,-0.145216,-0.218692,-0.031995,0.0
4,1.349860,0.766766,0.0,-0.859109,0.0,-0.026989,0.0,0.0,0.0,0.0,-0.707758,-0.145216,-0.218692,0.778826,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8135,1.208586,0.253004,0.0,0.062448,0.0,-0.026989,0.0,0.0,0.0,0.0,-0.707758,-0.145216,-0.218692,-0.031995,0.0
8136,0.225617,2.234657,0.0,0.615501,0.0,1.141576,0.0,0.0,0.0,0.0,-0.707758,1.319400,-0.218692,-2.464456,0.0
8137,1.274280,-1.068098,0.0,-0.561283,0.0,1.141576,0.0,0.0,0.0,0.0,-0.707758,-0.145216,-0.218692,-0.031995,0.0
8138,-0.913933,-0.407547,0.0,-1.438640,0.0,-0.416511,0.0,0.0,0.0,0.0,-0.707758,-0.145216,-0.218692,2.400467,0.0


In [6]:
svc=SVC()

In [7]:
param={
    "C":[1,2,3],
    "kernel":['rbf'],
    "gamma":[0.001,0.01],
    "tol":[0.0001,0.001],
    "cache_size":[200,300],
    "random_state":[33,100]
}

In [8]:
gscv=GridSearchCV(estimator=svc, param_grid=param)
gscv.fit(X_train,y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 2, 3], 'cache_size': [200, 300],
                         'gamma': [0.001, 0.01], 'kernel': ['rbf'],
                         'random_state': [33, 100], 'tol': [0.0001, 0.001]})

In [9]:
gscv.best_estimator_

SVC(C=3, gamma=0.01, random_state=33, tol=0.0001)

In [10]:
print(gscv.best_estimator_)

SVC(C=3, gamma=0.01, random_state=33, tol=0.0001)


In [11]:
print(gscv.best_estimator_)

SVC(C=3, gamma=0.01, random_state=33, tol=0.0001)


In [12]:
gscv.best_estimator_

SVC(C=3, gamma=0.01, random_state=33, tol=0.0001)